In [2]:
# System imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
from string import digits
import csv
import json
import numpy as np

pn.extension()

%matplotlib inline

In [3]:
# Local imports
import sys
sys.path.append("../lib2")

from Constants import Constants
from ProcessingTools import DateProcessingTools

In [4]:
# Build tools
debug_level = 0
constants = Constants()
tool_data_processing = DateProcessingTools(debug_level)

In [5]:
# Import all preprocessed data
atlas           = pd.read_pickle(constants.PREPROCESSED_ATLAS_FILE_PATH)
forecast        = pd.read_pickle(constants.PREPROCESSED_FORECAST_DATA_FILE_PATH)
revenue2020     = pd.read_pickle(constants.PREPROCESSED_REVENUE2020_FILE_PATH)
revenue2020A    = pd.read_pickle(constants.PREPROCESSED_REVENUE2020A_FILE_PATH)

In [ ]:
# Group by year and neighborhood and then create a new dataframe of the mean values
atlas.head()
# avg_price_df = atlas.groupby(by=['Invoice Amount', 'Customers'])
# avg_price_df.head()

In [21]:
# Use hvplot to create an interactive line chart of the average price per sq ft.
# The plot should have a dropdown selector for the neighborhood

avg_price_mean = avg_price_df.mean()

sfo_grouped_month_and_02_Anonymized_plot = avg_invoice_amount_mean.hvplot.line(
    x='customers',
    y='invoice ',
    xlabel= 'Customers',
    ylabel='Invoice Amount',
    groupby='Subscription',
    width=600
).opts(yformatter='%0f')

sfo_grouped_month_and_02_Anonymized_plot

DataError: No numeric types to aggregate